## Import Important Packages

In [ ]:
# import ipackages
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import pandas as pd
import os
from scipy.interpolate import interp1d, LinearNDInterpolator
import yt
from yt import load
from pyneb import Continuum
from astropy.cosmology import FlatLambdaCDM

# import functions from library folder
from library.calculate_quantities import *
from library.continuum_grid import *
from library.filter_tools import *
from library.prepare_data import *
from library.visualization import *
from library.yt_fields import *

## Useful Inputs

In [ ]:
# define useful inputs
input_path = "output_00273/info_00273.txt"
z = 10 # redshift
filter_file = "F200W_filter.txt"

# calculate distance of object for defined redshift
cosmo = FlatLambdaCDM(H0 = 70, Om0 = 0.3)
distance_Mpc = cosmo.angular_diameter_distance_z1z2(0, z)
distance_pc = distance_Mpc.to("pc").value

## Example: Single Filter Case (JWST Filter)

In [ ]:
# prepare simulation data
ds, ad, ctr, pop2, wl_shifted, trans, width = prepare_simulation_data(
    input_path = input_path,
    filter_path = filter_file,
    z = z,
    filter_dir = "JWST_filter_bins",
    wl_initial = 0.6,
    wl_final = 2.3,
    num_bins = 20
)

In [ ]:
# compute continuum grid
df_results, interp_funcs = compute_continuum_grid(
    min_temp = 1e3, max_temp = 1e5, num_temp_grid = 15,
    min_dense = 1e-4, max_dense = 1e5, num_dense_grid = 15,
    min_wl = 912, max_wl = 1e5, num_wl_grid = 1e5,
    filter_wl = wl_shifted,     # 1D array from prepare_simulation_data
    filter_output = trans,      # 1D array from prepare_simulation_data
    save_dir = "continuum_single"
)

In [ ]:
# add fliux fields to yt package
ds, filter_list = add_flux_fields(
    ds = ds, 
    interp_funcs = interp_funcs, 
    min_temp = 1e3, 
    max_temp = 1e7, 
    min_dense = 1e-4, 
    max_dense = 1e5,
    he_h_ratio = 0.1
)

In [ ]:
# visualize projection plot
create_projection_plot(
    ds = ds, 
    field_name = "flux_total", # options: "flux_contH", "flux_2p", "flux_ff", "flux_total" 
    ctr_at_code = ctr,        
    z = z,                    
    distance_pc = distance_pc,    
    plt_wdth = width,         
    plot_units = "jy_arcsec2", # options: "flux", "jy_arcsec2", "magnitude_arcsec2"
    axis_units = "arcsec", # options: "arcsec", "pc"
    filter_path = filter_file
)

In [ ]:
# visualize phase plot
create_phase_plot(
    ad = ad, 
    x_field = ("gas", "density"), 
    y_field = ("gas", "temperature"), 
    z_field = ("gas", "flux_total"),
    # x_bins = (1e-31, 1e-23),         
    # y_bins = (1e3, 1e7),             
    weight_field = None
)

**Note:** Spectra plot can be visualized for multiple filters only.

## Example: Multiple Filters Case

In [ ]:
# prepare simulation data
all_ds, all_ad, all_ctr, all_pop2, all_wl_shifted, all_trans, all_width = prepare_simulation_data(
    input_path = input_path,
    filter_path = None,
    z = z,
    filter_dir = "my_filter_bins",
    wl_initial = 0.6,
    wl_final = 2.3,
    num_bins = 20
)

In [ ]:
# compute continuum grid
all_dfs, all_interp_funcs = compute_continuum_grid(
    min_temp = 1e3, max_temp = 1e7, num_temp_grid = 15,
    min_dense = 1e-4, max_dense = 1e5, num_dense_grid = 15,
    min_wl = 912, max_wl = 1e5, num_wl_grid = 1e5,
    filter_wl = all_wl_shifted,     # 2D array (n_filters, wavelength_points)
    filter_output = all_trans,      # 2D array (n_filters, transmission_points)
    save_dir = "continuum_multi"
)

# check if your continuum grid is calculated correctly
print(df10.head())

In [ ]:
# add flux fields
all_ds, all_filter_list = add_flux_fields(
    ds = all_ds, 
    interp_funcs = all_interp_funcs, 
    min_temp = 1e3, 
    max_temp = 1e7, 
    min_dense = 1e-4, 
    max_dense = 1e5
)

**Note:** Projection plot and phase plot can be visualized for a single filter by default, so I only show what a projection plot and phase plot look like above. For multiple filters, you need to add the number of the filter you want to look at specifically.

In [ ]:
# visualize spectra plot
all_centers, all_y_vals, all_units = create_spectrum_plot(
    ds = all_ds, 
    filter_list = all_filter_list,
    z = z, 
    filter_dir = "filter_bins", 
    flux_type = "total",           # options: "total", "contH", "cont2p", "contff"
    plot_units = "jy_arcsec2"      # options: None, "mag_arcsec2", "jy_arcsec2"
)

plt.figure(figsize = (16, 8))
plt.plot(all_centers, all_y_vals, marker = 'o', linestyle = '-', linewidth = 2, markersize = 8, color = 'blue', markerfacecolor = 'white', markeredgewidth = 2)

# unit formatting for the label
y_label = f"Flux Density [{all_units}]" if all_units != "magnitude_arcsec2" else "Magnitude arcsec$^{-2}$"

plt.xlabel('Wavelength [Microns]', fontsize = 12)
plt.ylabel(y_label, fontsize = 12)
plt.title('Simulated Continuum Spectrum: Total Emission', fontsize = 14)

# log scale is recommended for all units except magnitude_arcsec2 to see faint features
if all_units != "magnitude_arcsec2":
    plt.yscale('log')

plt.grid(True, linestyle = '--', alpha = 0.6)
plt.show()